In [ ]:
import pandas as pd
from keras.utils import to_categorical
from sklearn.metrics import brier_score_loss
from IPython.display import SVG


def encodePhred(x):
    out = -10 * np.log10(1.00000000001-x)
    out = out.astype(np.int64)
    out[out > 60] = 60
    out[out <= 0] = 1
    return out

In [ ]:
def bow_model2():
    input_layer = 263
    output_layer = 2 

    h_layer1 = 512
    dropout1 = 0.25

    h_layer2 = 256
    dropout2 = 0.5

    h_layer3 = 128
    dropout3 = 0.5

    model = Sequential()

    model.add(Dense(h_layer1, activation='relu', input_shape=(input_layer, )))
    model.add(BatchNormalization())
    model.add(Dropout(dropout1))

    model.add(Dense(h_layer2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout2))
    
    model.add(Dense(h_layer3, activation='relu', ))
    model.add(BatchNormalization())

    model.add(Dense(output_layer, activation='softmax'))

    return model

In [ ]:
train_df = pd.read_csv("../data/train_gamcompare/csv/compared_mapped100_sim100..csv")

In [ ]:
train_df.head()

In [ ]:
y = to_categorical(train_df['correct'], 2)

In [ ]:
X = train_df.drop(columns=['correct', 'read', 'aligner'])

In [ ]:
X.shape

In [ ]:
model = bow_model2()

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [ ]:
model.fit(X, y, batch_size=128, epochs=1, verbose=1, shuffle=True)

In [ ]:
test_df = pd.read_csv("../data/test_gamcompare/csv/tcompared_tmapped100_tsim100..csv")

In [ ]:
y_train_pred = model.predict(X)

In [ ]:
brier_score_loss(train_df['correct'].values, y_train_pred[:, 1])

In [ ]:
brier_score_loss(train_df['correct'].values, train_df['mqp'].values)

In [ ]:
X_test = test_df.drop(columns=['correct', 'read', 'aligner'])

In [ ]:
y_test_pred = model.predict(X_test)

In [ ]:
brier_score_loss(test_df['correct'].values, y_test_pred[:, 1])

In [ ]:
def output_tsv(df, pred):
    out_df = df.loc[:, ['correct', 'mq', 'aligner', 'read']]
    recal_df =  out_df.copy()
    recal_df.loc[:, 'aligner'] = 'recal'
    recal_df.loc[:, 'mq'] = pred
    
    return out_df.append(recal_df, ignore_index=True)

In [ ]:
test_tsv = output_tsv(test_df, encodePhred(y_test_pred[:, 1]))

In [ ]:
test_tsv[test_tsv.aligner == "recal"].mq.hist(log=True)

In [ ]:
test_tsv.to_csv("bow_test.tsv", index=False, sep='\t')

In [ ]:
!Rscript ~/vg/scripts/plot-qq.R bow_test.tsv bow_test.svg

In [ ]:
!Rscript ~/vg/scripts/plot-roc.R bow_test.tsv roc-bow_test.svg

In [ ]:
SVG("bow_test.svg")

In [ ]:
SVG("roc-bow_test.svg")